In [16]:
import requests
import json
import os
from instabase.notebook.conf import ibconfig

# URI
INSTABASE_URI = 'https://www.instabase.com'
RUN_FLOW_API = INSTABASE_URI + '/api/v1/flow/run_flow_async'
GET_FLOW_STATUS_API = INSTABASE_URI + '/api/v1/jobs/status'

# Request header
MY_API_TOKEN = '<MY_API_TOKEN>'                                     # Fill this out: Put your API token here.
HEADERS = {
  'Authorization': 'Bearer {}'.format(MY_API_TOKEN),
}

def _pretty_print(d):
  try:
    d = json.loads(d)
  except:
    pass
  return json.dumps(d, sort_keys=True, indent=4, separators=(',', ': '))

def run_flow(input_dir_path, ibflow_file_path):
  data = json.dumps({
    'input_dir': input_dir_path,
    'ibflow_path': ibflow_file_path,
  })
  r = requests.post(RUN_FLOW_API, headers=HEADERS, data=data)
  return r.content

project_root_path = '<PATH_TO_PROJECT>'                             # Fill this out.
ibflow_file_path = '{}/<PATH_TO_FLOW>'.format(project_root_path)    # Fill this out.
input_dir_path = '{}/<PATH_TO_INPUT>'.format(project_root_path)     # Fill this out.

flow_resp = run_flow(input_dir_path, ibflow_file_path)
_pretty_print(flow_resp)

flow_resp_dict = json.loads(flow_resp)

In [15]:
import StringIO
import time

def get_status(job_id):
    r = requests.get(
        GET_FLOW_STATUS_API + '?job_id=' + job_id, headers=HEADERS, cookies=ibconfig.cookies)
    print(r.content)
    return json.loads(r.content)

def _display_file(full_path):
    
    _, ext = os.path.splitext(full_path)
    
    with ib.open('/' + full_path.lstrip('/'), 'r') as f:
        content = f.read()
    
    if ext == '.csv':
        display(pd.read_csv(StringIO.StringIO(content)))
    else:
        print(content)
    
def print_flow_result(flow_resp):
    print(flow_resp)
    job_id = None                                              # Fill this out: Extract the job_id from the flow_resp
    
    done = False
    resp = {}
    
    while not done:
        resp = get_status(job_id)
        print(resp)
        if resp['status'] != 'OK':
            print(resp['msg'])
            return

        if resp['state'] != 'DONE':
            cur_status = json.loads(resp.get('cur_status', '{}'))
            step_name = cur_status.get('stepName')
            msg = cur_status.get('curMsg')
            print('Running: ' + step_name + ', ' + msg + '...')
            time.sleep(4)
            continue
        
        done = True
        break
    
    output_folder = resp['results'][0]['output_folder']
    
    list_resp, err = ib.list_dir(output_folder)
    if err:
        print(err)
        return
    
    full_path = ''
    
    for node in list_resp['nodes']:
        if node['type'] != 'file':
            continue
        
        full_path = node['full_path']
        break
    
    if not full_path:
        print('No files found in the output directory')
        return
    _display_file(full_path)


print_flow_result(flow_resp_dict)

{u'msg': u'Authentication token not found. Please provide a valid auth token.', u'status': u'ERROR', u'statusCode': 400}


TypeError: cannot concatenate 'str' and 'NoneType' objects